# Configurações

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn import datasets
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

In [ ]:
n_splits = 10
n_repeats = 3
n_estimators = [3,9,15,21]

### Load dataset

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/VitorBonella/PL-Dataset/main/dataset.csv',sep=";") #Leitura dos dados para o pandas

Como minha matrícula é terminada em 9, utilizarei 7 descritores de Hu e 6 descritores de Haralick.

In [ ]:
# Lista de Indices
FOURIER = ['df01', 'df02', 'df03', 'df04','df05', 'df06', 'df07', 'df08', 'df09', 'df10']
HU = ['i1', 'i2', 'i3', 'i4','i5', 'i6', 'i7']
HARALICK = ['probmax', 'energia', 'entropia', 'contraste','homogeneidade', 'correlacao']
ALL = FOURIER + HU + HARALICK

df[ALL].head(2)

,df01,df02,df03,df04,df05,df06,df07,df08,df09,df10,...,i4,i5,i6,i7,probmax,energia,entropia,contraste,homogeneidade,correlacao
0,"0,0879606602603602","0,0540590780493764","0,0252096142058536","0,0274713661921584","0,0128341271697431","0,00837372424071006","0,00459165042660484","0,00516286747775679","0,00474694756923231","0,00307480942393677",...,"5,71775104737315e-005","3,80815942436415e-009","-1,41733745231029e-006","3,15336969940052e-009","0,890374128851521","3,17034178175013","6,5716176171224","-30,7129899022437","3,76304934586401","0,00030352446061056"
1,"0,0183325142757933","0,0641922380636373","0,0213351158490625","0,028934652413586","0,012263060875406","0,0112299899510848","0,00356611493895844","0,00586153710536664","0,00374648636964241","0,00377850078446776",...,"1,01157608366481e-008","-5,39593107944031e-015","-6,85966112547544e-011","3,90722879150443e-015","0,874335002692948","3,05605231787486","7,56143396285353","-37,1051952158663","3,72262223245045","0,000268692571829909"


In [ ]:
df.set_index('id',inplace=True) #Transformando a coluna id no indice da tabela

df[ALL] = df[ALL].apply(lambda x: x.str.replace(',', '.').astype(float), axis=1)

#Observem que a classe esta separada em duas colunas então devemos concatenadas para formar uma coluna só chamada classe
df['classe'] = df['tipo_lampada'].str.replace(" ", "") + df['potencia'].astype(str) 

In [ ]:
df['classe']

id
355    metalica400
356    metalica400
357    metalica400
358    metalica400
359    metalica400
          ...     
656    metalica250
657    metalica250
658    metalica250
659    metalica250
660    metalica250
Name: classe, Length: 297, dtype: object

In [ ]:
df[ALL].head(2)

,df01,df02,df03,df04,df05,df06,df07,df08,df09,df10,...,i4,i5,i6,i7,probmax,energia,entropia,contraste,homogeneidade,correlacao
id,,,,,,,,,,,,,,,,,,,,,
355,0.087961,0.054059,0.025210,0.027471,0.012834,0.008374,0.004592,0.005163,0.004747,0.003075,...,5.717751e-05,3.808159e-09,-1.417337e-06,3.153370e-09,0.890374,3.170342,6.571618,-30.712990,3.763049,0.000304
356,0.018333,0.064192,0.021335,0.028935,0.012263,0.011230,0.003566,0.005862,0.003746,0.003779,...,1.011576e-08,-5.395931e-15,-6.859661e-11,3.907229e-15,0.874335,3.056052,7.561434,-37.105195,3.722622,0.000269


In [ ]:
from sklearn.preprocessing import StandardScaler


X = df[ALL]
y = df['classe']


In [ ]:
def classification_report(scores):
    print(f'Media: {scores.mean():.8f}, Desvio Padrao: {scores.std():.8f}')
    inf, sup = stats.norm.interval(0.95, loc=scores.mean(), 
                               scale=scores.std()/np.sqrt(len(scores)))
    print(f'Intervalo de confiança (95%): [{inf:.8f},{sup:.8f}]')

## ZeroR

In [ ]:
from sklearn.dummy import DummyClassifier

zR = DummyClassifier()

rkf = RepeatedStratifiedKFold(n_splits = n_splits, n_repeats = n_repeats, random_state = 36851234)

scoresZR = cross_val_score(zR, X, y, cv = rkf)

classification_report(scoresZR)

Media: 0.16505747, Desvio Padrao: 0.01088258
Intervalo de confiança (95%): [0.16116326,0.16895168]


### Bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

parameters = {'bagging__n_estimators':n_estimators}

bg = BaggingClassifier(random_state=11)

pipe = Pipeline([('scaler', StandardScaler()), ('bagging', bg)])

clf = GridSearchCV(pipe, parameters, cv=4)

rkf = RepeatedStratifiedKFold(n_splits = n_splits, n_repeats = n_repeats, random_state = 36851234)

scoresBA = cross_val_score(clf, X, y, cv = rkf)

classification_report(scoresBA)

Media: 0.63563218, Desvio Padrao: 0.07451964
Intervalo de confiança (95%): [0.60896616,0.66229821]


### AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

parameters = {'boosting__n_estimators':n_estimators}

adb = AdaBoostClassifier(random_state=11)

pipe = Pipeline([('scaler', StandardScaler()), ('boosting',adb)])

clf = GridSearchCV(pipe, parameters,cv=4)

rkf = RepeatedStratifiedKFold(n_splits = n_splits, n_repeats = n_repeats, random_state = 36851234)

scoresAB = cross_val_score(clf, X, y, cv = rkf)

classification_report(scoresAB)

Media: 0.33904215, Desvio Padrao: 0.02916987
Intervalo de confiança (95%): [0.32860403,0.34948026]


### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

parameters = {'randomforest__n_estimators':n_estimators}

rF = RandomForestClassifier(random_state=11)

pipeline = Pipeline([('scaler', StandardScaler()), ('randomforest', rF)])

gs = GridSearchCV(pipeline, parameters, cv=4)

rkf = RepeatedStratifiedKFold(n_splits = n_splits, n_repeats = n_repeats, random_state = 36851234)

scoresRF = cross_val_score(gs, X, y, cv=rkf)

classification_report(scoresRF)

Media: 0.62314176, Desvio Padrao: 0.07457384
Intervalo de confiança (95%): [0.59645635,0.64982718]


### Heterogeneus Pooling

In [ ]:
from sklearn.base import BaseEstimator
from sklearn.utils import resample
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils import check_random_state
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, RepeatedStratifiedKFold
from sklearn.preprocessing import StandardScaler
import numpy as np
from collections import Counter


class HeterogeneousPooling(BaseEstimator, ClassifierMixin):
    def __init__(self, n_samples=1):
        self.n_samples = n_samples
        self.classifiers = []

    def fit(self, X, y):
        class_counts = Counter(y)
        self.class_order_ = sorted(class_counts, key=class_counts.get, reverse=True)
        random_state = 0

        dt = DecisionTreeClassifier()
        nb = GaussianNB()
        knn = KNeighborsClassifier()

        for i in range(self.n_samples):
            if i == 0:
                X_train = X
                y_train = y
            else:
                random_state += 1
                X_train, y_train = resample(X, y, replace=True, random_state=random_state)


            clf_dt = dt.fit(X_train, y_train)
            clf_nb = nb.fit(X_train, y_train)
            clf_knn = knn.fit(X_train, y_train)

            self.classifiers.append(clf_dt)
            self.classifiers.append(clf_nb)
            self.classifiers.append(clf_knn)

    def predict(self, X):
        predictions = np.column_stack([clf.predict(X) for clf in self.classifiers])

        final_predictions = []
        for i in range(predictions.shape[0]):
            votes = predictions[i, :]

            vote_count = Counter(votes)
            max_vote = max(vote_count.values())

            most_frequent_class = None
            for cls, count in vote_count.items():
                if count == max_vote:
                    if most_frequent_class is None:
                        most_frequent_class = cls
                    elif self.class_order_.index(cls) < self.class_order_.index(most_frequent_class):
                        most_frequent_class = cls

            final_predictions.append(most_frequent_class)

        return final_predictions


In [ ]:
parameters = {'heterogeneous__n_samples': [1, 3, 5, 7]}

hp = HeterogeneousPooling()

pipe = Pipeline([('scaler', StandardScaler()), ('heterogeneous', hp)])

clf = GridSearchCV(pipe, parameters, cv=4)

rkf = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=36851234)

HP_scores = cross_val_score(clf, X, y, cv=rkf)
classification_report(HP_scores)
HP_scores

Media: 0.56126437, Desvio Padrao: 0.09820611
Intervalo de confiança (95%): [0.52612241,0.59640633]


array([0.66666667, 0.26666667, 0.4       , 0.6       , 0.56666667,
       0.56666667, 0.56666667, 0.5862069 , 0.44827586, 0.65517241,
       0.5       , 0.43333333, 0.56666667, 0.5       , 0.5       ,
       0.7       , 0.53333333, 0.5862069 , 0.62068966, 0.5862069 ,
       0.66666667, 0.66666667, 0.63333333, 0.73333333, 0.6       ,
       0.46666667, 0.56666667, 0.44827586, 0.65517241, 0.55172414])